In [1]:
from time import time
import numpy as np
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer, InputSpec, Dense, Input
from tensorflow.python.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras import callbacks
from tensorflow.python.keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import metrics
from custom_layers import autoencoder as auto_encoder
from custom_layers import ClusteringLayer
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import cv2
import os, glob, shutil

In [2]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [3]:
x_train = [cv2.resize(cv2.imread(file), (224, 224)) for file in glob.glob("input/images/*.png")]
#x_test = [cv2.resize(cv2.imread(file), (224, 224)) for file in glob.glob("images/test/images/*.png")]
print("Images loaded")

Images loaded


In [4]:
n_clusters = 5  # No. of clusters
x = np.concatenate(x_train)
x = x.reshape((x.shape[0], -1))
x = np.divide(x, 255.)

In [5]:
# dim[0] = og dimensions (w * h) dim[1:] = layers of auto encoder. 
# Numbers equal the shape the encoder forces the data into in the layer.
dims = [x.shape[-1], 500, 500, 2000, 10]
init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')
pretrain_optimizer = SGD(learning_rate=1, momentum=0.9)
pretrain_epochs = 50
batch_size = 800
save_dir = './weights'

In [6]:
autoencoder, encoder = auto_encoder(dims, init=init)
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
autoencoder.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs)  # , callbacks=cb)
autoencoder.save_weights(save_dir + '/ae_weights.h5')

Epoch 1/50
280/280 [==============================] - 20s 69ms/step - loss: 0.0341
Epoch 2/50
280/280 [==============================] - 20s 71ms/step - loss: 0.0133
Epoch 3/50
280/280 [==============================] - 20s 72ms/step - loss: 0.0120
Epoch 4/50
280/280 [==============================] - 20s 71ms/step - loss: 0.0101
Epoch 5/50
280/280 [==============================] - 20s 73ms/step - loss: 0.0075
Epoch 6/50
280/280 [==============================] - 33s 117ms/step - loss: 0.0052
Epoch 7/50
280/280 [==============================] - 19s 69ms/step - loss: 0.0039
Epoch 8/50
280/280 [==============================] - 20s 70ms/step - loss: 0.0031
Epoch 9/50
280/280 [==============================] - 19s 70ms/step - loss: 0.0025
Epoch 10/50
280/280 [==============================] - 19s 69ms/step - loss: 0.0021
Epoch 11/50
280/280 [==============================] - 20s 70ms/step - loss: 0.0018
Epoch 12/50
280/280 [==============================] - 20s 70ms/step - loss: 0.0016


In [7]:
autoencoder.save_weights(save_dir + '/ae_weights.h5')

In [8]:
autoencoder.load_weights(save_dir + '/ae_weights.h5')

In [9]:
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=clustering_layer)
model.compile(optimizer=SGD(0.01, 0.9), loss='kld')

In [10]:
kmeans = KMeans(n_clusters=n_clusters, n_init=100, verbose=True, max_iter = 1000)
y_pred = kmeans.fit_predict(encoder.predict(x))
y_pred_last = np.copy(y_pred)
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 12551535.0
start iteration
done sorting
end inner loop
Iteration 1, inertia 11866196.0
start iteration
done sorting
end inner loop
Iteration 2, inertia 11664239.0
start iteration
done sorting
end inner loop
Iteration 3, inertia 11600899.0
start iteration
done sorting
end inner loop
Iteration 4, inertia 11490077.0
start iteration
done sorting
end inner loop
Iteration 5, inertia 11448576.0
start iteration
done sorting
end inner loop
Iteration 6, inertia 11419826.0
start iteration
done sorting
end inner loop
Iteration 7, inertia 11387280.0
start iteration
done sorting
end inner loop
Iteration 8, inertia 11371210.0
start iteration
done sorting
end inner loop
Iteration 9, inertia 11326620.0
start iteration
done sorting
end inner loop
Iteration 10, inertia 11296795.0
start iteration
done sorting
end inner loop
Iteration 11, inertia 11145005.0
start iteration
done sorting
end inner loop
Iteration 12, iner

end inner loop
Iteration 13, inertia 11189110.0
start iteration
done sorting
end inner loop
Iteration 14, inertia 11042414.0
start iteration
done sorting
end inner loop
Iteration 15, inertia 10682681.0
start iteration
done sorting
end inner loop
Iteration 16, inertia 10523880.0
start iteration
done sorting
end inner loop
Iteration 17, inertia 10473947.0
start iteration
done sorting
end inner loop
Iteration 18, inertia 10462218.0
start iteration
done sorting
end inner loop
Iteration 19, inertia 10457796.0
start iteration
done sorting
end inner loop
Iteration 20, inertia 10454044.0
start iteration
done sorting
end inner loop
Iteration 21, inertia 10450994.0
start iteration
done sorting
end inner loop
Iteration 22, inertia 10448980.0
start iteration
done sorting
end inner loop
Iteration 23, inertia 10448193.0
start iteration
done sorting
end inner loop
Iteration 24, inertia 10447991.0
start iteration
done sorting
end inner loop
Iteration 25, inertia 10447928.0
center shift 5.509200e-04 wi

Iteration 3, inertia 10936928.0
start iteration
done sorting
end inner loop
Iteration 4, inertia 10890789.0
start iteration
done sorting
end inner loop
Iteration 5, inertia 10789080.0
start iteration
done sorting
end inner loop
Iteration 6, inertia 10741274.0
start iteration
done sorting
end inner loop
Iteration 7, inertia 10729346.0
start iteration
done sorting
end inner loop
Iteration 8, inertia 10720803.0
start iteration
done sorting
end inner loop
Iteration 9, inertia 10707530.0
start iteration
done sorting
end inner loop
Iteration 10, inertia 10695762.0
start iteration
done sorting
end inner loop
Iteration 11, inertia 10674515.0
start iteration
done sorting
end inner loop
Iteration 12, inertia 10667926.0
start iteration
done sorting
end inner loop
Iteration 13, inertia 10664497.0
start iteration
done sorting
end inner loop
Iteration 14, inertia 10662843.0
start iteration
done sorting
end inner loop
Iteration 15, inertia 10641828.0
start iteration
done sorting
end inner loop
Iterat

Iteration 10, inertia 10659682.0
center shift 3.409825e-04 within tolerance 9.983878e-04
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 12480619.0
start iteration
done sorting
end inner loop
Iteration 1, inertia 11353689.0
start iteration
done sorting
end inner loop
Iteration 2, inertia 10824445.0
start iteration
done sorting
end inner loop
Iteration 3, inertia 10739750.0
start iteration
done sorting
end inner loop
Iteration 4, inertia 10710717.0
start iteration
done sorting
end inner loop
Iteration 5, inertia 10699071.0
start iteration
done sorting
end inner loop
Iteration 6, inertia 10692710.0
start iteration
done sorting
end inner loop
Iteration 7, inertia 10691358.0
start iteration
done sorting
end inner loop
Iteration 8, inertia 10690992.0
start iteration
done sorting
end inner loop
Iteration 9, inertia 10690802.0
start iteration
done sorting
end inner loop
Iteration 10, inertia 10690770.0
center shift 3.270199e-04 within tolerance 9.98387

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 11814896.0
start iteration
done sorting
end inner loop
Iteration 1, inertia 11183430.0
start iteration
done sorting
end inner loop
Iteration 2, inertia 10962451.0
start iteration
done sorting
end inner loop
Iteration 3, inertia 10707289.0
start iteration
done sorting
end inner loop
Iteration 4, inertia 10562807.0
start iteration
done sorting
end inner loop
Iteration 5, inertia 10486151.0
start iteration
done sorting
end inner loop
Iteration 6, inertia 10419595.0
start iteration
done sorting
end inner loop
Iteration 7, inertia 10403936.0
start iteration
done sorting
end inner loop
Iteration 8, inertia 10399119.0
start iteration
done sorting
end inner loop
Iteration 9, inertia 10397423.0
start iteration
done sorting
end inner loop
Iteration 10, inertia 10397002.0
start iteration
done sorting
end inner loop
Iteration 11, inertia 10396909.0
center shift 6.153482e-04 within tolerance 9.983878e-04
Initia

...

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 12639951.0
start iteration
done sorting
end inner loop
Iteration 1, inertia 11775316.0
start iteration
done sorting
end inner loop
Iteration 2, inertia 11244227.0
start iteration
done sorting
end inner loop
Iteration 3, inertia 10810083.0
start iteration
done sorting
end inner loop
Iteration 4, inertia 10578707.0
start iteration
done sorting
end inner loop
Iteration 5, inertia 10516638.0
start iteration
done sorting
end inner loop
Iteration 6, inertia 10509040.0
start iteration
done sorting
end inner loop
Iteration 7, inertia 10492935.0
start iteration
done sorting
end inner loop
Iteration 8, inertia 10456720.0
start iteration
done sorting
end inner loop
Iteration 9, inertia 10446987.0
start iteration
done sorting
end inner loop
Iteration 10, inertia 10445126.0
start iteration
done sorting
end inner loop
Iteration 11, inertia 10444245.0
start iteration
done sorting
end inner loop
Iteration 12, iner

Iteration 16, inertia 10699412.0
start iteration
done sorting
end inner loop
Iteration 17, inertia 10699373.0
center shift 6.011410e-04 within tolerance 9.983878e-04
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 11977438.0
start iteration
done sorting
end inner loop
Iteration 1, inertia 10832497.0
start iteration
done sorting
end inner loop
Iteration 2, inertia 10644181.0
start iteration
done sorting
end inner loop
Iteration 3, inertia 10609815.0
start iteration
done sorting
end inner loop
Iteration 4, inertia 10594882.0
start iteration
done sorting
end inner loop
Iteration 5, inertia 10553748.0
start iteration
done sorting
end inner loop
Iteration 6, inertia 10420650.0
start iteration
done sorting
end inner loop
Iteration 7, inertia 10408888.0
start iteration
done sorting
end inner loop
Iteration 8, inertia 10408107.0
start iteration
done sorting
end inner loop
Iteration 9, inertia 10407719.0
start iteration
done sorting
end inner loop
Iterat

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 12116114.0
start iteration
done sorting
end inner loop
Iteration 1, inertia 11170681.0
start iteration
done sorting
end inner loop
Iteration 2, inertia 10996377.0
start iteration
done sorting
end inner loop
Iteration 3, inertia 10972389.0
start iteration
done sorting
end inner loop
Iteration 4, inertia 10966706.0
start iteration
done sorting
end inner loop
Iteration 5, inertia 10965285.0
start iteration
done sorting
end inner loop
Iteration 6, inertia 10960278.0
start iteration
done sorting
end inner loop
Iteration 7, inertia 10952949.0
start iteration
done sorting
end inner loop
Iteration 8, inertia 10949837.0
start iteration
done sorting
end inner loop
Iteration 9, inertia 10949357.0
start iteration
done sorting
end inner loop
Iteration 10, inertia 10949248.0
start iteration
done sorting
end inner loop
Iteration 11, inertia 10949207.0
center shift 5.965930e-04 within tolerance 9.983878e-04
Initia

Iteration 7, inertia 10611933.0
start iteration
done sorting
end inner loop
Iteration 8, inertia 10600905.0
start iteration
done sorting
end inner loop
Iteration 9, inertia 10599677.0
start iteration
done sorting
end inner loop
Iteration 10, inertia 10599556.0
start iteration
done sorting
end inner loop
Iteration 11, inertia 10599535.0
center shift 2.220116e-04 within tolerance 9.983878e-04
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 12068207.0
start iteration
done sorting
end inner loop
Iteration 1, inertia 11200192.0
start iteration
done sorting
end inner loop
Iteration 2, inertia 10950178.0
start iteration
done sorting
end inner loop
Iteration 3, inertia 10751525.0
start iteration
done sorting
end inner loop
Iteration 4, inertia 10624921.0
start iteration
done sorting
end inner loop
Iteration 5, inertia 10499929.0
start iteration
done sorting
end inner loop
Iteration 6, inertia 10468482.0
start iteration
done sorting
end inner loop
Iterat

Iteration 8, inertia 10448850.0
start iteration
done sorting
end inner loop
Iteration 9, inertia 10448174.0
start iteration
done sorting
end inner loop
Iteration 10, inertia 10447980.0
start iteration
done sorting
end inner loop
Iteration 11, inertia 10447927.0
center shift 5.114224e-04 within tolerance 9.983878e-04
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 12551976.0
start iteration
done sorting
end inner loop
Iteration 1, inertia 10862183.0
start iteration
done sorting
end inner loop
Iteration 2, inertia 10536941.0
start iteration
done sorting
end inner loop
Iteration 3, inertia 10497754.0
start iteration
done sorting
end inner loop
Iteration 4, inertia 10495998.0
start iteration
done sorting
end inner loop
Iteration 5, inertia 10493732.0
start iteration
done sorting
end inner loop
Iteration 6, inertia 10483916.0
start iteration
done sorting
end inner loop
Iteration 7, inertia 10442599.0
start iteration
done sorting
end inner loop
Iterat

In [11]:
model.save_weights(save_dir + '/DEC_model_final.h5')

In [12]:
model.load_weights(save_dir + '/DEC_model_final.h5')

In [13]:
paths = glob.glob("images/train/images/*.png")
print("paths found")
for i in range(n_clusters):
    if os.path.exists("output/cluster" + str(i)):
        shutil.rmtree("output/cluster" + str(i))
        os.makedirs("output/cluster" + str(i))
    else:
        os.makedirs("output/cluster" + str(i))
    print("output/cluster" + str(i))
print("Moving Images")
for i in range(len(paths)):
    shutil.copy2(paths[i], "output/cluster"+str(y_pred_last[i]))
    print(str(i+1) + "/" + str(len(paths)) + " Images Copied")

paths found
output/cluster0
output/cluster1
output/cluster2
output/cluster3
output/cluster4
Moving Images
1/801 Images Copied
2/801 Images Copied
3/801 Images Copied
4/801 Images Copied
5/801 Images Copied
6/801 Images Copied
7/801 Images Copied
8/801 Images Copied
9/801 Images Copied
10/801 Images Copied
11/801 Images Copied
12/801 Images Copied
13/801 Images Copied
14/801 Images Copied
15/801 Images Copied
16/801 Images Copied
17/801 Images Copied
18/801 Images Copied
19/801 Images Copied
20/801 Images Copied
21/801 Images Copied
22/801 Images Copied
23/801 Images Copied
24/801 Images Copied
25/801 Images Copied
26/801 Images Copied
27/801 Images Copied
28/801 Images Copied
29/801 Images Copied
30/801 Images Copied
31/801 Images Copied
32/801 Images Copied
33/801 Images Copied
34/801 Images Copied
35/801 Images Copied
36/801 Images Copied
37/801 Images Copied
38/801 Images Copied
39/801 Images Copied
40/801 Images Copied
41/801 Images Copied
42/801 Images Copied
43/801 Images Copied


376/801 Images Copied
377/801 Images Copied
378/801 Images Copied
379/801 Images Copied
380/801 Images Copied
381/801 Images Copied
382/801 Images Copied
383/801 Images Copied
384/801 Images Copied
385/801 Images Copied
386/801 Images Copied
387/801 Images Copied
388/801 Images Copied
389/801 Images Copied
390/801 Images Copied
391/801 Images Copied
392/801 Images Copied
393/801 Images Copied
394/801 Images Copied
395/801 Images Copied
396/801 Images Copied
397/801 Images Copied
398/801 Images Copied
399/801 Images Copied
400/801 Images Copied
401/801 Images Copied
402/801 Images Copied
403/801 Images Copied
404/801 Images Copied
405/801 Images Copied
406/801 Images Copied
407/801 Images Copied
408/801 Images Copied
409/801 Images Copied
410/801 Images Copied
411/801 Images Copied
412/801 Images Copied
413/801 Images Copied
414/801 Images Copied
415/801 Images Copied
416/801 Images Copied
417/801 Images Copied
418/801 Images Copied
419/801 Images Copied
420/801 Images Copied
421/801 Im

772/801 Images Copied
773/801 Images Copied
774/801 Images Copied
775/801 Images Copied
776/801 Images Copied
777/801 Images Copied
778/801 Images Copied
779/801 Images Copied
780/801 Images Copied
781/801 Images Copied
782/801 Images Copied
783/801 Images Copied
784/801 Images Copied
785/801 Images Copied
786/801 Images Copied
787/801 Images Copied
788/801 Images Copied
789/801 Images Copied
790/801 Images Copied
791/801 Images Copied
792/801 Images Copied
793/801 Images Copied
794/801 Images Copied
795/801 Images Copied
796/801 Images Copied
797/801 Images Copied
798/801 Images Copied
799/801 Images Copied
800/801 Images Copied
801/801 Images Copied


In [14]:
for 
#x_train = [cv2.resize(cv2.imread(file), (224, 224)) for file in glob.glob("output/cluster0/*.png")]